In [1]:
import glob
import gzip
from bs4 import BeautifulSoup
import lxml
from collections import defaultdict
from tqdm import tqdm

path = '/nfs/gns/literature/rdf_annotation_data/daily_pipeline_api/15_08_1947/fulltext/job_1/annotation/'

In [2]:
all_files = sorted(glob.glob(path+'*.xml.gz'))

In [3]:
len(all_files)

30

In [4]:
all_files[0]

'/nfs/gns/literature/rdf_annotation_data/daily_pipeline_api/15_08_1947/fulltext/job_1/annotation/patch-total-0.xml.gz'

In [5]:
def getfileblocks(file_path):

    subFileBlocks = []

    with gzip.open(file_path, 'rt') as fh:
        for line in fh:
            if line.startswith('<!DOCTYPE article'):
                subFileBlocks.append(line)
            else:
                subFileBlocks[-1] += line
                
    return subFileBlocks

In [6]:
def GP_soup(file_soup):
    dict_sentences = defaultdict(list)
    for each_ztag in file_soup.find_all('z:uniprot'):
        try:
            gene_sentence = each_ztag.findParents('plain')[0].text
            dict_sentences[gene_sentence].append(each_ztag) 
        except:
            pass
    return dict_sentences


from fuzzywuzzy import fuzz 
def compare_with_ml_annotations(r, z_tags, ml_json):
    FP_removed_z_tags = []
    for each_z_tag in z_tags:
        for each_ml_annotation in r.json()['annotations']:
            score = fuzz.token_set_ratio(each_ml_annotation[3],each_z_tag.text)
            if  score ==100 and each_ml_annotation[2]=='GP':
                FP_removed_z_tags.append(each_z_tag)
    
    return list(set(z_tags) - set(FP_removed_z_tags))



In [7]:
import requests

def get_unique_fp_tags(GP_dict_sentences):
    url = 'http://ai-capo-api-2:5200/spaCy_ner_predictor?text_sentence='

    FP_dict_sentences =defaultdict(list)
    set_FPs = set()
    
    for each_uniprot_sentence, z_tags in GP_dict_sentences.items():
        r = requests.get(url+each_uniprot_sentence)
        if r.status_code == 200 and r.json()['status']==200:
            FPs = compare_with_ml_annotations(r, z_tags,r.json())
            if FPs:
                FP_dict_sentences[each_uniprot_sentence].append(FPs)

        
    for FP_sent,FP_tag in FP_dict_sentences.items():
        for each_ztag in FP_tag[0]:
            set_FPs.add(each_ztag)

    return list(set_FPs)
        

In [8]:
path_new = '/nfs/gns/literature/machine-learning/Santosh/fp_removed_patches/'
# for each_file_content in subFileBlocks:
for each_file_path in all_files:
    ss = getfileblocks(each_file_path)
    for each_article in tqdm(ss):
        soup = BeautifulSoup(each_article, 'xml')
        GP_dict_sentences = GP_soup(soup)
        if GP_dict_sentences:
            unique_fps = get_unique_fp_tags(GP_dict_sentences)

            for each_fp in unique_fps:
                for tag in soup.findAll('z:uniprot', attrs={'ids':each_fp['ids']}):
                    tag.unwrap()

            new_file_content = str(soup).replace('<?xml version="1.0" encoding="utf-8"?>\n','').replace('</ebiroot>','</ebiroot>\n')

            with gzip.open(path_new+'edited_'+each_file_path.split('/')[-1], 'at') as f:
                f.write(new_file_content)



 43%|████▎     | 433/1000 [07:47<13:00,  1.38s/it]

KeyboardInterrupt: 

In [9]:
# Test for bsub

import subprocess
from subprocess import Popen, PIPE
import glob


full_text_path = '/nfs/gns/literature/rdf_annotation_data/daily_pipeline_api/15_08_1947/fulltext/'
all_jobs_path = sorted(glob.glob(full_text_path + 'job_*'))

xml_annotation_path = []
for each_job_path in tqdm(all_jobs_path):
    xml_annotation_path.extend(sorted(glob.glob(each_job_path + '/annotation/' + '*.xml.gz')))
    
    


100%|██████████| 193/193 [00:00<00:00, 424.88it/s]


In [10]:
xml_annotation_path[0]

'/nfs/gns/literature/rdf_annotation_data/daily_pipeline_api/15_08_1947/fulltext/job_1/annotation/patch-total-0.xml.gz'

In [18]:
job_cmd = '/nfs/gns/literature/Santosh_Tirunagari/Submissions/python_run.job.capo.fp.removal.sh'
MyOut = subprocess.Popen([job_cmd, xml_annotation_path[0]],
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT)
stdout,stderr = MyOut.communicate()
print(stdout)
print(stderr)

b'Job <154116> is submitted to default queue <research-rh74>.\n'
None


In [39]:

abs(2-len(str(stdout).split('\\n')))%39

5

In [40]:
import time
time.sleep(600)
print('hello')

hello


In [ ]:
/nfs/gns/literature/machine-learning/Capo_full_test_ML_filtering\

In [ ]:
# offset = 25

# start_index = key_fp.find(value_fp[0][0].text)
# end_index = start_index+len(value_fp[0][0].text)

# key_fp[start_index:end_index]

# exact = value_fp[0][0].text
# prefix = key_fp[max(0,start_index-offset):start_index-1]
# postfix = key_fp[end_index+1:end_index+offset]
            
# [prefix, exact, postfix]      

# # make sure the index starts from 0, if the start_index-offset<0, set it 0
# # the total offset incuding prefix and posfix is 50
# 

# value_fp[0][0].attrs['ids'].split(',')[0]
# normalisation = 'http://purl.uniprot.org/uniprot/'+value_fp[0][0].attrs['ids'].split(',')[0]
# normalisation